In [1]:
import dlib
import dlib
import cv2
from imutils import face_utils
import argparse
from utils import *
import os
import glob
import torch

In [71]:
DATAPATH = 'write you path'
WALK = os.walk(DATAPATH)

In [6]:
list_path_fake = []
for (a,b,c) in WALK:
    for _c in c:
        if '.jpg' in _c or '.png' in _c:
            list_path_fake.append(os.path.join(a,_c))
list_path_fake.sort()
print(len(list_path_fake))

In [14]:
def parse_args():
    desc = "Data preprocessing for Deep3DRecon."
    parser = argparse.ArgumentParser(description=desc)

    parser.add_argument('--img_path', type=str, default='./input', help='original images folder')
    parser.add_argument('--save_path', type=str, default='./processed_data', help='custom path to save proccessed images and labels')


    return parser.parse_args()

def midpoint(p1, p2):
    coords = (p1[0] + p2[0]) / 2, (p1[1] + p2[1]) / 2
    return [int(x) for x in coords]


In [ ]:

def preprocessing(list_path_fake, find_main_point = False):
    dict_landmark = {}
    img_list = list_path_fake

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

    for file in img_list:
        img = cv2.imread(file) #img must be cropped into face
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)

        if len(rects) > 0:
            for rect in rects:
                if find_main_point:
                    x = rect.left()
                    y = rect.top()
                    w = rect.right()
                    h = rect.bottom()
                    img = img[y:h, x:w]

                shape = predictor(img, rect)
                shape_np = face_utils.shape_to_np(shape).tolist()
                if find_main_point:
                    left_eye = midpoint(shape_np[36], shape_np[39])
                    right_eye = midpoint(shape_np[42], shape_np[45])
                    features = [left_eye, right_eye, shape_np[33], shape_np[48], shape_np[54]]
                    dict_landmark[file] = features
                else:
                    dict_landmark[file] = shape_np
    return dict_landmark


# Get Landmarks

In [ ]:
dict_land = {}
dict_land = preprocessing(list_path_fake)
len(dict_land)

# Translation DICT -> JSON 

In [10]:
import json
with open('landmark_test.json','w', encoding='utf-8') as make_file:
    json.dump(dict_land,make_file,indent='\t')

In [ ]:
import json
with open('landmark_test.json', 'r') as f:
    landmarks_record =  json.load(f)
len(landmarks_record)